# <span style="color:darkblue"> Lecture 15 - Merging Data </span>

<font size = "5">

In the previous class we covered ...

- Aggregate Statistics
- Merge aggregate stats

In this class we will cover ...

- More database merging!
- Emphasize importance of cleaning before merging
- Database concatenation

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [11]:
results_raw  = pd.read_csv("data_raw/results.csv")
races_raw    = pd.read_csv("data_raw/races.csv")
circuits_raw = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Multi-file datasets can be visualized with an ...

- "Entity Relationship Diagram" (ERD)
- How the identifiers in each table are connected
- Complement to the "codebook"

<img src="figures/erd_f1_simple.png" alt="drawing" width="600"/>


<font size = "5">

Start by opening datasets!

- Check columns with similar names

In [12]:
# We extract all the unique values in races_raw["name"] and circuits_raw["name"]
# We use "sort_values()" to make it easier to compare the variables
# The "codebook/f1_codebook.pdf" file shows that the content is indeed different

unique_data_races    = pd.unique(races_raw["name"].sort_values())
unique_data_circuits = pd.unique(circuits_raw["name"].sort_values())


# <span style="color:darkblue"> II. Dictionaries + Renaming </span>

<font size = "5">

A dictionary is another way to store data. 

- Defined with curly brackets "{...}"
- Different fields are separated by a comma
- Assign values to a field with a colon ":"

<font size = "5">

Dictionaries + Pandas

In [13]:
# This is an example of a pandas data frame created from a dictionary
# This example illustrates the basic syntax of a dictionary

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year":      ["2018","2023","2022"] }

pd.DataFrame(car_dictionary)


,car_model,year
0,Ferrari,2018
1,Tesla,2023
2,BMW,2022


<font size = "5">

Rename columns with dictionaries

``` {"old_name": "new_name"} ```

In [14]:
# We first define the dictionary
# Change the pipe ".rename(...)" to rename the columns
# Dictionaries can flexibly accommodate single values or list after ":"

dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits_raw.rename(columns = dict_rename_circuits)


<font size = "5">
Check that ".rename()" worked

In [15]:
# Extract the column names of the "raw" and "clean" data

print("Old List:")
print(circuits_raw.columns.values)
print("")
print("New List:")
print(circuits.columns.values)


Old List:
['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

New List:
['circuitId' 'circuitRef' 'circuit_name' 'location' 'country' 'lat' 'lng'
 'alt' 'url']


<font size = 5>

Try it yourself!

- Create a dictionary to rename "name" to "race_name"
- Rename this column in the "races_raw" dataset
- Store the output in a new dataset called "races"

In [16]:
# Write your own code

dict_rename_races= {"name" : "race_name"}
races = races_raw.rename(columns = dict_rename_races)

print("Old List:")
print(races_raw.columns.values)
print("")
print("New List:")
print(races.columns.values)

Old List:
['raceId' 'year' 'round' 'circuitId' 'name' 'date' 'time' 'url' 'fp1_date'
 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time' 'quali_date'
 'quali_time' 'sprint_date' 'sprint_time']

New List:
['raceId' 'year' 'round' 'circuitId' 'race_name' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time']


# <span style="color:darkblue"> II. Merging </span>


<font size = "5">

Extracting specific columns from dataset

In [17]:
circuits[["circuitId","circuit_name"]]

,circuitId,circuit_name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


<font size = "5">

Merge datasets

<img src="figures/merge_goal.png" alt="drawing" width="500"/>


```pd.merge(data1,data2,on,how)```

- Strive to merge only specific columns of data2
- Avoid merging all columns
- Keeping it simple gives you more control over the output

In [32]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset (much include the "on" column)
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(races_raw,
                       circuits[["circuitId","circuit_name"]],
                       on = "circuitId",
                       how = "left")

display(races_merge[["raceId", "circuitId", "circuit_name"]].sort_values(by="circuit_name"))

,raceId,circuitId,circuit_name
760,761,61,AVUS
434,435,29,Adelaide Street Circuit
335,336,29,Adelaide Street Circuit
319,320,29,Adelaide Street Circuit
370,371,29,Adelaide Street Circuit
...,...,...,...
501,502,40,Zolder
486,487,40,Zolder
470,471,40,Zolder
516,517,40,Zolder


In [31]:
# Another example of merging

results_merge = pd.merge(results_raw,
                         races_raw[["raceId","date"]],
                         on = "raceId",
                         how = "left")

display(results_merge[['resultId', 'raceId', 'date']].sort_values(by ='date'))

,resultId,raceId,date
20034,20035,833,1950-05-13
20044,20045,833,1950-05-13
20035,20036,833,1950-05-13
20043,20044,833,1950-05-13
20042,20043,833,1950-05-13
...,...,...,...
25821,25827,1096,2022-11-20
25820,25826,1096,2022-11-20
25838,25844,1096,2022-11-20
25828,25834,1096,2022-11-20


<font size = "5">
<span style="color:red"> Common pitfall: </span> What happens if you don't rename?

In [20]:
# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(races_raw,
                               circuits_raw[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)

print(races_merge_pitfall.columns.values)


['raceId' 'year' 'round' 'circuitId' 'name_x' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'name_y']


<font size = "5">

Try it yourself!

- Rename the columns "name_x" and "name_y" <br>
in the dataset "races_merge_pitfall" to <br>
 "race_name" and "circuit_name"

$\quad$ HINT: Create a dictionary and use "pd.rename()"

In [21]:
# Write your own code

dict_rename_pitfall = {"name_x": "race_name",
                       "name_y": "circuit_name"}

races_merge_pitfall = races_merge_pitfall.rename(columns = dict_rename_pitfall)

print(races_merge_pitfall.columns.values)

['raceId' 'year' 'round' 'circuitId' 'race_name' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'circuit_name']


<font size = "5">

Try it yourself!

- Merge the column "alt", "lng", and "lat" into the races data <br>
using "pd.merge()

In [22]:
# Write your own code
circuits_merge = pd.merge(races_raw,
                         circuits_raw[["circuitId", "alt","lng","lat"]],
                         on = "circuitId",
                         how = "left")

circuits_merge.columns.values


array(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time',
       'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date',
       'fp3_time', 'quali_date', 'quali_time', 'sprint_date',
       'sprint_time', 'alt', 'lng', 'lat'], dtype=object)

# <span style="color:darkblue"> III. Concat </span>


<font size = "5">

Use ".query()" to split data into different parts

In [23]:
circuits_spain = circuits.query('country == "Spain"')
circuits_usa   = circuits.query('country == "United States" | country == "USA"')
circuits_malaysia = circuits.query('country == "Malaysia"')

<font size = "5">

Cocatenate data back together

- Useful if there are datasets split by geography...
- year, or other subgroup

In [24]:
# Works best if columns are identical
# There are also other advanced options if they are not 
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html

circuits_concat = pd.concat([circuits_spain,circuits_usa])
display(circuits_concat)

,circuitId,circuitRef,circuit_name,location,country,lat,lng,alt,url
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.5700,2.261110,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
11,12,valencia,Valencia Street Circuit,Valencia,Spain,39.4589,-0.331667,4,http://en.wikipedia.org/wiki/Valencia_Street_C...
25,26,jerez,Circuito de Jerez,Jerez de la Frontera,Spain,36.7083,-6.034170,37,http://en.wikipedia.org/wiki/Circuito_Permanen...
44,45,jarama,Jarama,Madrid,Spain,40.6171,-3.585580,609,http://en.wikipedia.org/wiki/Circuito_Permanen...
48,49,montjuic,Montjuïc,Barcelona,Spain,41.3664,2.151670,79,http://en.wikipedia.org/wiki/Montju%C3%AFc_cir...
66,67,pedralbes,Circuit de Pedralbes,Barcelona,Spain,41.3903,2.116670,85,http://en.wikipedia.org/wiki/Pedralbes_Circuit
18,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.234700,223,http://en.wikipedia.org/wiki/Indianapolis_Moto...
22,80,vegas,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173000,\N,https://en.wikipedia.org/wiki/Las_Vegas_Grand_...
32,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.075000,345,http://en.wikipedia.org/wiki/Phoenix_street_ci...
36,37,detroit,Detroit Street Circuit,Detroit,USA,42.3298,-83.040100,177,http://en.wikipedia.org/wiki/Detroit_street_ci...


<font size = "5">

Try it yourself!

- Concatenate the USA and Malaysia datasets



In [25]:
# Write your own code

circuits_concatenate = pd.concat([circuits_usa, circuits_malaysia])
display(circuits_concatenate)




,circuitId,circuitRef,circuit_name,location,country,lat,lng,alt,url
18,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.79500,-86.2347,223,http://en.wikipedia.org/wiki/Indianapolis_Moto...
22,80,vegas,Las Vegas Strip Street Circuit,Las Vegas,United States,36.11470,-115.1730,\N,https://en.wikipedia.org/wiki/Las_Vegas_Grand_...
32,33,phoenix,Phoenix street circuit,Phoenix,USA,33.44790,-112.0750,345,http://en.wikipedia.org/wiki/Phoenix_street_ci...
36,37,detroit,Detroit Street Circuit,Detroit,USA,42.32980,-83.0401,177,http://en.wikipedia.org/wiki/Detroit_street_ci...
41,42,dallas,Fair Park,Dallas,USA,32.77740,-96.7587,139,http://en.wikipedia.org/wiki/Fair_Park
42,43,long_beach,Long Beach,California,USA,33.76510,-118.1890,12,"http://en.wikipedia.org/wiki/Long_Beach,_Calif..."
43,44,las_vegas,Las Vegas Street Circuit,Nevada,USA,36.11620,-115.1740,639,http://en.wikipedia.org/wiki/Las_Vegas_Street_...
45,46,watkins_glen,Watkins Glen,New York State,USA,42.33690,-76.9272,485,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...
59,60,riverside,Riverside International Raceway,California,USA,33.93700,-117.2730,470,http://en.wikipedia.org/wiki/Riverside_Interna...
62,63,sebring,Sebring International Raceway,Florida,USA,27.45470,-81.3483,18,http://en.wikipedia.org/wiki/Sebring_Raceway
